# Dataset preprocessing

The goal of this notebook is to create a preprocessed kaggle dataset out of the competition dataset.  
For now, the preprocessing will be based on [this notebook](https://www.kaggle.com/code/vonmainstein/imu-tof).  
It consists of the following steps:
-   Set the appropriate dtypes (helps with RAM usage).
-   Impute missing feature values with forward, backward and then 0 filling.
-   Split the dataset into multiple cross validation folds.
-   Standardize feature values.
-   Pad/Truncate the sequences to the same length.  

> Note:  
> - Demographics data set will be ignored for now.  

## Imports

In [1]:
import os
import json
from os.path import join
from itertools import repeat, starmap

import numpy as np
import pandas as pd
from numpy import ndarray
import plotly.express as px
from pandas import DataFrame as DF
from scipy.spatial.transform import Rotation
from kagglehub import whoami, competition_download, dataset_upload

from config import *

## Data preprocessing

### Load dataset
Requires to be logged in if this notebook is not running on kaggle, go to [your settings](https://www.kaggle.com/settings) to create an access token and put it in `~/.kaggle/`.

In [2]:
competition_dataset_path = competition_download(COMPETITION_HANDLE)
df = pd.read_csv(join(competition_dataset_path, "train.csv"), dtype=DATASET_DF_DTYPES)

### Impute missing data
Perform forward, backward and then 0 filling of NaN values.

In [3]:
feature_cols = list(set(df.columns) - set(META_DATA_COLUMNS))
# Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
# So we replace them by NaN and then perform imputing.  
tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}

def imputed_features(df:DF) -> DF:
    # Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
    # So we replace them by NaN and then perform imputing.  
    tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
    fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}

    df[feature_cols] = (
        df
        .loc[:, feature_cols]
        # df.replace with np.nan sets dtype to floar64 so we set it back to float32
        .replace(tof_vals_to_nan, value=np.nan)
        .astype("float32")
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .ffill()
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .bfill()
        # In case there are only nan in the column in the sequence
        .fillna(fillna_val_per_col)
    )
    return df
    
df = imputed_features(df)

### Norm quaternions
This is usefull for using them with a Rotation instance.

In [4]:
QUATERNION_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
def norm_quat_rotations(df:DF) -> DF:
    df[QUATERNION_COLS] /= np.linalg.norm(df[QUATERNION_COLS], axis=1, keepdims=True)
    return df

### Add gravity free acceleration features

In [ ]:
GRAVITY_WORLD = np.array([0, 0, 9.81])
ACCELRATION_COLS = ["acc_x", "acc_y", "acc_z"]
GRAVITY_FREE_ACC_COLS = ["gravity_free_" + col for col in ACCELRATION_COLS]
def add_gravity_free_acc_cols(df:DF) -> DF:
    # Vectorized version of https://www.kaggle.com/code/wasupandceacar/lb-0-82-5fold-single-bert-model#Dataset `remove_gravity_from_acc`
    rotations = Rotation.from_quat(df[QUATERNION_COLS])
    gravity_sensor_frame = rotations.apply(GRAVITY_WORLD, inverse=True)
    df[GRAVITY_FREE_ACC_COLS] = df[ACCELRATION_COLS] - gravity_sensor_frame
    return df

df = add_gravity_free_acc_cols(df)

### Euler angles from quaternions

In [5]:
EULER_ANGLES_COLS = ["euler_x", "euler_y", "euler_z"]
def rot_euler_angles(seq:DF) -> ndarray:
    try:
        quat_cols = seq[QUATERNION_COLS]
        rotation = Rotation.from_quat(quat_cols)
        euler_data = rotation.as_euler("xyz").squeeze()
        angles_df = DF(
            data=euler_data,
            columns=EULER_ANGLES_COLS
        )
        return angles_df
    except ValueError as e:
        print(quat_cols)
        raise e

rot_euler_angles_df = (
    df
    .groupby("sequence_id", as_index=False, observed=True)
    .apply(rot_euler_angles, include_groups=False)
    .loc[:, EULER_ANGLES_COLS]
    .values
)
display(rot_euler_angles_df)
df[EULER_ANGLES_COLS] = rot_euler_angles_df

### One hot encode target values.

In [7]:
one_hot_target = pd.get_dummies(df["gesture"])
df[one_hot_target.columns] = one_hot_target
df

/tmp/ipykernel_75726/1507406683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[one_hot_target.columns] = one_hot_target
/tmp/ipykernel_75726/1507406683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[one_hot_target.columns] = one_hot_target
/tmp/ipykernel_75726/1507406683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Neck - scratch,Text on phone,Wave hello,Write name in air,Write name on leg,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,False,False,False,False,False
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,False,False,False,False,False
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,False,False,False,False,False
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,False,False,False,False,False
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,True,False,False,False,False,False
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,True,False,False,False,False,False
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,True,False,False,False,False,False
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,True,False,False,False,False,False


### ToF data aggregation.
Time of Flight columns take most of the data, let's reduce their size by aggregating by mean for each Time of Flight sensor.

In [8]:
def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in range(1, 6):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = (
            df
            # Need to use a dict otherwise the name of the col will be "tof_preffix" instead of the value it contains
            .assign(**{tof_name:df[tof_cols].mean(axis="columns")})
            .drop(columns=tof_cols)
        )
    return df

df = agg_tof_cols_per_sensor(df)
# Redifine feature_cols now that there are less of them.
feature_cols = list(set(df.columns) - set(META_DATA_COLUMNS) - set(df["gesture"].unique().tolist()))

df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off,tof_1,tof_2,tof_3,tof_4,tof_5
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,139.250000,117.109375,91.687500,123.359375,135.343750
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,139.796875,119.671875,97.921875,124.406250,137.000000
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,142.375000,128.359375,116.953125,125.687500,140.234375
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,154.109375,142.093750,144.515625,149.078125,142.609375
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,177.953125,149.453125,161.828125,163.765625,151.265625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,False,68.562500,67.750000,144.437500,74.062500,52.843750
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,False,70.234375,66.656250,144.000000,70.406250,54.531250
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,False,66.671875,62.906250,136.906250,70.109375,57.468750
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,False,67.218750,64.500000,140.531250,75.609375,54.937500


### Split into folds

In [9]:
def split_dataset(df:DF, by="subject") -> tuple[DF, DF]:
    unique_sequences = df[by].unique()
    validation_sequences = pd.Series(unique_sequences).sample(
        frac=VALIDATION_FRACTION, replace=False
    )

    validation_set = df[df[by].isin(validation_sequences)]
    train_set = df[~df[by].isin(validation_sequences)]

    return train_set, validation_set

folds = list(map(split_dataset, repeat(df, N_FOLDS)))

In [10]:
display(df.shape)
display(folds[0][0].shape)
display(folds[0][1].shape)

(574945, 50)

(456902, 50)

(118043, 50)

In [11]:
for train_df, validation_df in folds:
    print(train_df.shape, validation_df.shape)

(456902, 50) (118043, 50)
(457817, 50) (117128, 50)
(460665, 50) (114280, 50)
(460960, 50) (113985, 50)
(454964, 50) (119981, 50)


### Std norm
Standard scale the feature cols (should probably do something different for IMU cols).  
<!-- *Deprecated, std norm is now performed at dataset creation to avoid target leakage.*   -->

In [12]:
def std_norm_dataset(train:DF, val:DF) -> tuple[DF, DF]:
    means = train[feature_cols].mean()
    std = train[feature_cols].std()
    train.loc[:, feature_cols] = (train[feature_cols] - means) / std
    val.loc[:, feature_cols] = (val[feature_cols] - means) / std
    return train, val

normed_folds = list(starmap(std_norm_dataset, folds))

/tmp/ipykernel_75726/2075072895.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.55618001  1.55302021  1.5556055  ... -1.0006715  -0.89955795
 -0.89438737]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  train.loc[:, feature_cols] = (train[feature_cols] - means) / std
/tmp/ipykernel_75726/2075072895.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.43246421 0.46172138 0.47183199 ... 0.84278109 0.85775242 0.86470371]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  train.loc[:, feature_cols] = (train[feature_cols] - means) / std
/tmp/ipykernel_75726/2075072895.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.04369583 -0.03719973 -0.03265245 ... -1.60055136 -1.60587817
 -

Normalize full dataset.

In [13]:
# Retain full dataset meta data for inference
full_dataset_meta_data = {
    "mean": df[feature_cols].mean().to_dict(),
    "std": df[feature_cols].std().to_dict(),
}
df.loc[:, feature_cols] = (df[feature_cols] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']

/tmp/ipykernel_75726/4279755229.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.01677826  0.00259618  0.02630819 ... -1.00227249 -0.90048435
 -0.89527928]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.loc[:, feature_cols] = (df[feature_cols] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']
/tmp/ipykernel_75726/4279755229.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.36724552 0.38668742 0.42327279 ... 0.84895603 0.86489636 0.87229757]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.loc[:, feature_cols] = (df[feature_cols] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']
/tmp/ipykernel_75726/4279755229.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future e

In [14]:
df[feature_cols].agg(["mean", "std"])

,tof_4,gravity_free_acc_z,thm_4,rot_x,acc_x,acc_z,thm_1,gravity_free_acc_x,gravity_free_acc_y,tof_3,...,rot_z,rot_w,rot_y,tof_1,tof_2,acc_y,euler_y,euler_x,euler_z,thm_5
mean,1.404808e-08,-2.372822e-18,-1.427808e-07,-2.663767e-08,-1.749467e-08,-2.971537e-09,3.772558e-08,2.293728e-17,-8.858535e-17,2.663846e-09,...,1.855411e-08,-6.315844e-08,-8.728876e-10,1.908273e-07,1.687596e-08,1.186949e-08,1.581881e-18,1.581881e-17,-4.982926e-17,-3.565034e-09
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Let's compare the train to validation mean/std skews.

In [15]:
pd.concat({
    "train": folds[0][1][feature_cols].agg(["mean", "std"]),
    "validation": folds[0][1][feature_cols].agg(["mean", "std"]),
})

tof_4  gravity_free_acc_z     thm_4     rot_x     acc_x  \
train      mean -0.024555            0.028677  0.221284 -0.070501  0.121474   
           std   0.967049            1.007557  0.622032  0.937755  0.962222   
validation mean -0.024555            0.028677  0.221284 -0.070501  0.121474   
           std   0.967049            1.007557  0.622032  0.937755  0.962222   

                    acc_z     thm_1  gravity_free_acc_x  gravity_free_acc_y  \
train      mean  0.062355  0.077026            0.128376            0.054878   
           std   1.014286  0.734685            0.966044            0.953626   
validation mean  0.062355  0.077026            0.128376            0.054878   
           std   1.014286  0.734685            0.966044            0.953626   

                    tof_3  ...     rot_z     rot_w     rot_y     tof_1  \
train      mean  0.046934  ... -0.165708  0.071373 -0.083885  0.048334   
           std   0.989750  ...  0.944784  0.982139  0.988288  0.962760   
validation mean  0.046934  ... -0.165708  0.071373 -0.083885  0.048334   
           std   0.989750  ...  0.944784  0.982139  0.988288  0.962760   

                   tof_2     acc_y   euler_y   euler_x   euler_z     thm_5  
train      mean  0.10647 -0.018677  0.132814 -0.104257  0.022463  0.241085  
           std   0.94748  0.978780  0.971776  0.978552  0.963772  0.445276  
validation mean  0.10647 -0.018677  0.132814 -0.104257  0.022463  0.241085  
           std   0.94748  0.978780  0.971776  0.978552  0.963772  0.445276  

[4 rows x 23 columns]

### Normalize sequences lengths.  
And turn the Dataframes into ndarrays.

#### Visualize histogram of sequences lengths.

Entire dataset sequences lengths.

In [16]:
px.histogram(
    (
        df
        .groupby("sequence_id", observed=True)
        .size()
    ),
    title="Sequence length frequency",
)

Second(to avoid always look at the first one) Train/validation split sequences lengths comparaison.

In [17]:
def get_set_sequences_lengths(set:DF, name:str) -> DF:
    return (
        set
        .groupby("sequence_id", observed=True)
        .size()
        .reset_index(name="length")
        .assign(set=name)
    )

full_se_lengths = pd.concat((
    get_set_sequences_lengths(folds[2][0], "Train"),
    get_set_sequences_lengths(folds[2][1], "Validation"),
))

fig = px.histogram(
    full_se_lengths,
    x="length",
    color="set",
    barmode="overlay",  # or 'group' if you want side-by-side bars
    nbins=50,           # adjust bin size if needed
    title="Sequence Length Distribution: Train vs Validation"
)

fig.update_traces(opacity=0.8)  # better visibility with overlay
fig.show()


In [18]:
for train, val in folds:
    print("train normed sequence len:", int(train.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print("validation normed sequence len:", int(val.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print()

train normed sequence len: 122
validation normed sequence len: 151

train normed sequence len: 124
validation normed sequence len: 136

train normed sequence len: 125
validation normed sequence len: 132

train normed sequence len: 126
validation normed sequence len: 129

train normed sequence len: 123
validation normed sequence len: 139



#### Sequence length norm implementation

In [19]:
from tqdm.notebook import tqdm

gesture_cols = df["gesture"].unique()

def length_normed_sequence_feat_arr(sequence: DF, normed_sequence_len: int) -> ndarray:
    features = (
        sequence
        .loc[:, feature_cols]
        .values
    )
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def df_to_ndarrays(df:DF, normed_sequence_len:int) -> tuple[np.ndarray, np.ndarray]:
    sequence_it = df.groupby("sequence_id", observed=True, as_index=False)
    x = np.empty(
        shape=(len(sequence_it), normed_sequence_len, len(feature_cols)),
        dtype="float32"
    )
    y = np.empty(
        shape=(len(sequence_it), df["gesture"].nunique()),
        dtype="float32"
    )
    for sequence_idx, (_, sequence) in tqdm(enumerate(sequence_it), total=len(sequence_it)):
        normed_seq_feat_arr = length_normed_sequence_feat_arr(sequence, normed_sequence_len)
        x[sequence_idx] = normed_seq_feat_arr
        # Take the first value as they are(or at least should be) all the same in a single sequence
        y[sequence_idx] = sequence[gesture_cols].iloc[0].values

    return x, y

def get_normed_seq_len(dataset:DF) -> int:
    return int(
        dataset
        .groupby("sequence_id", observed=True)
        .size()
        .quantile(SEQUENCE_NORMED_LEN_QUANTILE)
    )

def fold_dfs_to_ndarrays(train:DF, validation:DF) -> tuple[ndarray, ndarray, ndarray, ndarray]:
    """
    Returns:
        (train X, train Y, validation X, validation Y)
    """
    normed_sequence_len = get_normed_seq_len(train)
    return (
        *df_to_ndarrays(train, normed_sequence_len),
        *df_to_ndarrays(validation, normed_sequence_len),
    )

folds_arrs = list(starmap(fold_dfs_to_ndarrays, folds))

  0%|          | 0/6567 [00:00<?, ?it/s]

  0%|          | 0/1584 [00:00<?, ?it/s]

  0%|          | 0/6567 [00:00<?, ?it/s]

  0%|          | 0/1584 [00:00<?, ?it/s]

  0%|          | 0/6520 [00:00<?, ?it/s]

  0%|          | 0/1631 [00:00<?, ?it/s]

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

In [20]:
full_dataset_sequence_length_norm = get_normed_seq_len(df)
full_x, full_y = df_to_ndarrays(df, full_dataset_sequence_length_norm)

  0%|          | 0/8151 [00:00<?, ?it/s]

## Create dataset

In [21]:
# Clean dataset directory if it already exists
! rm -rf preprocessed_dataset
# Create dataset direcory
! mkdir preprocessed_dataset
# Save folds
for fold_i, (train_x, train_y, val_x, val_y) in enumerate(folds_arrs):
    fold_dir_path = join("preprocessed_dataset", f"fold_{fold_i}")
    os.makedirs(fold_dir_path)
    # save features (X)
    np.save(join(fold_dir_path, "train_X.npy"), train_x, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_X.npy"), val_x, allow_pickle=False)
    # Save targets (Y)
    np.save(join(fold_dir_path, "train_Y.npy"), train_y, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_Y.npy"), val_y, allow_pickle=False)
# Save full dataset
full_dataset_dir_path = "preprocessed_dataset/full_dataset"
os.makedirs(full_dataset_dir_path)
np.save(join(full_dataset_dir_path, "X.npy"), full_x, allow_pickle=False)
np.save(join(full_dataset_dir_path, "Y.npy"), full_y, allow_pickle=False)
# Save dataset meta data
full_dataset_meta_data["target_names"] = one_hot_target.columns.to_list()
full_dataset_meta_data["pad_seq_len"] = full_dataset_sequence_length_norm
full_dataset_meta_data["feature_cols"] = feature_cols

with open("preprocessed_dataset/full_dataset_meta_data.json", "w") as fp:
    json.dump(full_dataset_meta_data, fp, indent=4)

In [22]:
df[feature_cols]

,tof_4,gravity_free_acc_z,thm_4,rot_x,acc_x,acc_z,thm_1,gravity_free_acc_x,gravity_free_acc_y,tof_3,...,rot_z,rot_w,rot_y,tof_1,tof_2,acc_y,euler_y,euler_x,euler_z,thm_5
0,-0.016778,-0.191143,0.367246,-0.508525,0.872408,0.625816,0.502318,0.921929,0.378283,-0.454938,...,-1.238540,-0.997151,-0.716343,0.339571,0.148727,0.884130,0.883372,0.191240,0.339185,0.477938
1,0.002596,-0.239701,0.386687,-0.476437,0.918353,0.588012,0.593105,0.935808,0.441917,-0.337148,...,-1.267920,-0.957737,-0.681840,0.349948,0.199531,0.884130,0.860138,0.150843,0.297587,0.492298
2,0.026308,0.018062,0.423273,-0.334147,0.706192,0.964766,0.821468,0.779645,0.663317,0.022423,...,-1.349747,-0.630264,-0.549352,0.398864,0.371769,0.723320,0.754947,-0.047236,0.140441,0.510858
3,0.459197,0.236127,0.484648,-0.312448,0.858218,1.134561,0.868207,0.839486,0.700636,0.543179,...,-1.390175,-0.290093,-0.329928,0.621508,0.644067,0.347835,0.714206,-0.231340,-0.051374,0.341522
4,0.731018,0.639598,0.592139,-0.214344,0.679165,1.655481,0.587718,0.600054,0.487647,0.870276,...,-1.447599,-0.134027,-0.007333,1.073910,0.789974,-0.302433,0.418963,-0.342103,-0.252579,0.334854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,-0.929112,-0.166781,0.854270,-1.601608,0.322408,-1.309212,0.590561,0.345242,-1.541394,0.541703,...,0.070326,-1.117506,-0.759754,-1.001627,-0.829872,-0.444509,0.287935,1.457859,-1.902655,0.656701
574941,-0.996778,-0.249065,0.870208,-1.607920,0.369030,-1.434796,0.599804,0.390242,-1.547135,0.533437,...,0.045559,-1.100313,-0.740360,-0.969906,-0.851557,-0.490566,0.326326,1.471826,-1.902869,0.618153
574942,-1.002272,-0.024379,0.848956,-1.605422,0.249436,-1.138775,0.620099,0.333282,-1.418637,0.399410,...,0.041674,-1.089203,-0.741939,-1.037499,-0.925905,-0.314143,0.335326,1.470549,-1.901584,0.618153
574943,-0.900484,-0.223986,0.864896,-1.610813,0.402138,-1.414933,0.695652,0.433050,-1.473273,0.467900,...,0.017150,-1.070422,-0.722432,-1.027123,-0.894307,-0.429677,0.374099,1.484620,-1.901015,0.561656


## Dataset upload
Optionally upload the dataset to kaggle.

In [23]:
if input("Do you want to upload the  dataset to kaggle?[yes/no]").lower() == "yes":
    # Updaload the dataset
    dataset_upload(
        join(whoami()["username"], "prepocessed-cmi-2025"),
        "preprocessed_dataset",
        version_notes="Preprocessed Child Mind Institue 2025 competition dataset."
    )
else:
    print("Dataset has not been uploaded.")

Dataset has not been uploaded.
